In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import requests
from io import StringIO
import requests
import time
from selenium.webdriver import Chrome
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import os
from naver_crawler import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import re
import parser
import datetime
from urllib.request import urlopen
import webbrowser
import xmltodict
import json
from ast import literal_eval

<ipython-input-1-e9ca380508bf>:23: DeprecationWarning: The parser module is deprecated and will be removed in future versions of Python
  import parser


In [2]:
# 한국기업평가 설립일 : 1983.12.29
# 1980 ~ 2021 date_list 만들기
import time
from datetime import date,timedelta
today= date.today()
date_lst=[]
for i in range(459):
    date_lst.append((today - timedelta(days=30*i)).strftime("%Y-%m-%d"))
len(date_lst),date_lst[-1],date_lst[0]

(459, '1983-12-08', '2021-07-21')

In [7]:
table_save = table_DF.copy()

In [11]:
table_DF = pd.DataFrame()
type_lst = ['회사채','기업어음','전자단기사채','ABS','ABCP','ABSTB','ICR','IFSR']
os.getcwd()

'C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project'

In [15]:
# 창 띄우기
delay=3
browser = Chrome()
browser.implicitly_wait(delay)

In [16]:
url = 'http://www.rating.co.kr/disclosure/QDisclosure014.do'
browser.get(url)

In [17]:
# 1달이라는 기간동안
for t in range(len(date_lst)-1):
    start_date = browser.find_element_by_xpath('//*[@id="fromDt"]')
    start_date.clear()
    start_date.send_keys(date_lst[t+1])
    end_date = browser.find_element_by_xpath('//*[@id="toDt"]')
    end_date.clear()
    end_date.send_keys(date_lst[t])
    
    # 8종류의 데이터를
    for i in range(2,10):
        
        type_btn = browser.find_element_by_xpath('//*[@id="selectTitle"]/option[{}]'.format(i))
        type_btn.click()

        src_btn = browser.find_element_by_xpath('//*[@id="content"]/div[1]/div/div/ul/li[2]/a/span')
        src_btn.click()
        
        # 1,11,21,... 페이지
        for k in range(10):
            

            # 1페이지 추출
            result = browser.page_source
            soup = BeautifulSoup(result, 'html.parser')
            table = soup.select('div.table_type4')
            table_html = str(table)

            table_df = pd.read_html(table_html)[0]
            table_df['Type']=type_lst[i] # Type : 1~8 가지의 종류의 데이터 이름
            table_DF=pd.concat([table_DF,table_df],axis=0)
            print("table_DF:",table_DF.shape)

            # 2~10페이지 추출
            for j in range(4,13):
                
                try:
                    next_btn = browser.find_element_by_css_selector('#paging > a:nth-child({})'.format(j))
                    next_btn.click()
                    
                    result = browser.page_source
                    soup = BeautifulSoup(result, 'html.parser')
                    table = soup.select('div.table_type4')
                    table_html = str(table)

                    # 1. 회사채
                    table_df = pd.read_html(table_html)[0]
                    table_df['Type']=type_lst[i]
                    print("table_df",table_df.head())
                    table_DF=pd.concat([table_DF,table_df],axis=0)
                    print("table_DF:",table_DF.shape)
                except:
                    break
                    
                
                    
            # 다음 페이지(11,21...)로 넘어가기
            print(10*k,"페이지 완료 | table_DF.shape: ",table_DF.shape)
            print(table_DF)
            
            try:
                nextpg_btn = browser.find_element_by_css_selector('#paging > a.next')
                nextpg_btn.click()
            except:
                break
        

table_DF: (20, 12)
table_df           회사명         종류  구분                      회차   발행액 직전등급(Outlook)  \
0  (주)DGB금융지주  HB(Cocos)  정기   조건부자본증권(상)1(신종-영구-5콜)  1500      AA-(안정적)   
1  (주)DGB금융지주  HB(Cocos)  정기  조건부자본증권(상) 3(신종-영구-5콜)   500      AA-(안정적)   
2  (주)DGB금융지주  HB(Cocos)  정기  조건부자본증권(상) 2(신종-영구-5콜)  1000      AA-(안정적)   
3  (주)DGB금융지주         FB  정기                     9-2   500      AAA(안정적)   
4  (주)DGB금융지주         FB  정기                     8-2   600      AAA(안정적)   

  현재등급(Outlook)         평가일                     공시일  평가의견  보고서    Type  
0     AA- (안정적)  2021.06.24  2021.06.24 ( 15 : 25 )   NaN  NaN  전자단기사채  
1     AA- (안정적)  2021.06.24  2021.06.24 ( 15 : 25 )   NaN  NaN  전자단기사채  
2     AA- (안정적)  2021.06.24  2021.06.24 ( 15 : 25 )   NaN  NaN  전자단기사채  
3     AAA (안정적)  2021.06.24  2021.06.24 ( 15 : 25 )   NaN  NaN  전자단기사채  
4     AAA (안정적)  2021.06.24  2021.06.24 ( 15 : 25 )   NaN  NaN  전자단기사채  
table_DF: (40, 12)
table_df          회사명              종류  구분    회차   발행액 직전등급

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="selectTitle"]/option[3]"}
  (Session info: chrome=91.0.4472.164)


In [18]:
table_DF.shape

(80, 12)

In [9]:
table_DF.drop_duplicates().shape

(280, 12)

In [120]:
src_btn = browser.find_element_by_xpath('//*[@id="content"]/div[1]/div/div/ul/li[2]/a/span')
src_btn.click()

In [122]:
try:
    nextpg_btn = browser.find_element_by_css_selector('#paging > a.next')
    nextpg_btn.click()
except:
    pass

In [198]:
table_DF1.평가일.str.slice(0,4).value_counts()

2020    2838
2021    2328
2019    2222
2018    1450
2017    1108
2016     670
2015     524
2014     268
2013     210
2012     138
2011      60
2010      10
Name: 평가일, dtype: int64

In [205]:
table_DF1=table_DF1.drop_duplicates()
table_DF2=table_DF2.drop_duplicates()
table_DF3=table_DF3.drop_duplicates()
table_DF4=table_DF4.drop_duplicates()
table_DF5=table_DF5.drop_duplicates()
table_DF6=table_DF6.drop_duplicates()
table_DF7=table_DF7.drop_duplicates()
table_DF8=table_DF8.drop_duplicates()

In [209]:
print("table_DF1:",table_DF1.shape)
print("table_DF2:",table_DF2.shape)
print("table_DF3:",table_DF3.shape)
print("table_DF4:",table_DF4.shape)
print("table_DF5:",table_DF5.shape)
print("table_DF6:",table_DF6.shape)
print("table_DF7:",table_DF7.shape)
print("table_DF8:",table_DF8.shape)

table_DF1: (11620, 11)
table_DF2: (299, 9)
table_DF3: (114, 10)
table_DF4: (119, 11)
table_DF5: (25, 11)
table_DF6: (5216, 11)
table_DF7: (420, 12)
table_DF8: (16, 12)


In [212]:
os.chdir('C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project\\1_Data')
table_DF1.to_csv('KOREA_CREDIT_1.csv',index=False)
table_DF2.to_csv('KOREA_CREDIT_2.csv',index=False)
table_DF3.to_csv('KOREA_CREDIT_3.csv',index=False)
table_DF4.to_csv('KOREA_CREDIT_4.csv',index=False)
table_DF5.to_csv('KOREA_CREDIT_5.csv',index=False)
table_DF6.to_csv('KOREA_CREDIT_6.csv',index=False)
table_DF7.to_csv('KOREA_CREDIT_7.csv',index=False)
table_DF8.to_csv('KOREA_CREDIT_8.csv',index=False)

In [213]:
os.getcwd()

'C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project\\1_Data'